# Загрузка "потока событий" в Redis

In [7]:
from typing import Dict, List

import redis
import json

import pandas as pd


class Constants:
    USER_ID = "user_id"
    ITEM_ID = "item_id"
    ITEMS_IDS = "items_ids"
    TIMESTAMP = "time"

    TRANSACTIONS_PATH = "/Users/alfa/Documents/diplom/graphnn-recommendation-system/data/processed_transactions_train.csv"


def load_data_to_redis(user_sequences: List[Dict], r: redis.Redis) -> None:
    """
    Загрузка данных в Redis
    """
    for user_sequence in user_sequences:
        user_id = user_sequence.get(Constants.USER_ID)
        items_ids = user_sequence.get(Constants.ITEMS_IDS)

        r.set(f"user:{user_id}", json.dumps(items_ids))

    print("Данные успешно загружены в Redis")


# Подключение к Redis
r = redis.Redis(
    host="localhost",
    password="redis123",
    port=6379,
    decode_responses=True,
)

In [8]:
# Загрузка предобработанных транзакций
transactions = pd.read_parquet(Constants.TRANSACTIONS_PATH)

# Подготовка последовательностей для каждого пользователя
user_sequences = (
    transactions.groupby([Constants.USER_ID], as_index=False)[Constants.ITEM_ID]
    .apply(list)
    .rename(columns={Constants.ITEM_ID: Constants.ITEMS_IDS})
    .to_dict("records")
)

# Загрузка данных в Redis
load_data_to_redis(user_sequences=user_sequences, r=r)

Данные успешно загружены в Redis


In [9]:
# Проверка данных для случайного пользователя
user_id = "f1e14e6af00322b44591c0ca1f52f9993e425efb7dd2f83458c0ee22ff12c8ab"
data = r.get(f"user:{user_id}")

if data:
    items = json.loads(data)  # Десериализация из JSON
    print(f"Товары пользователя {user_id}: {items}")
else:
    print("Ключ не найден")

Товары пользователя f1e14e6af00322b44591c0ca1f52f9993e425efb7dd2f83458c0ee22ff12c8ab: [495685001, 569974001, 569974001, 448509017, 684890001, 684890001, 690276002, 683476001, 683476001, 534181003, 683476002, 683476002, 549914003, 549914003, 694671001, 694671001, 720812001, 668537002, 710879001, 710879001, 647982002, 670687004, 670687004, 656686002, 725327001, 725327001, 691839001, 691839001, 297067002, 595773002, 671632001, 707535001, 769400003, 679514001, 679514001, 692930003, 689109003, 726546001, 671974001, 377436002, 484205016, 484205016, 607154001, 726546001, 671974001, 484205016, 607154001, 671326001, 484205016, 484205016, 671974001, 671974001, 671974001, 695457003, 709187001, 710611001, 663498002, 663498002, 663498002, 678963016, 678963016, 484205004, 484205004, 765123001, 779883001, 779883001, 738236002, 738236002, 678694001, 738236002, 715927002, 708336001, 747637005, 738236002, 763159002, 763159002, 708855005, 708855005, 763159001, 763159001, 707215001, 708638001, 720438002, 

In [10]:
import requests

response = requests.get(
    "http://localhost:8000/recommendations/personal_items/",
    params={
        "user_id": "f1e14e6af00322b44591c0ca1f52f9993e425efb7dd2f83458c0ee22ff12c8ab"
    },
)
print(response.json())

{'recommendations': [{'item_id': '123', 'score': 0.5}, {'item_id': '123', 'score': 0.5}, {'item_id': '123', 'score': 0.5}]}
